<a href="https://colab.research.google.com/github/epowell101/mscGNN-work/blob/main/EDA_and_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Required Libraries
import pandas as pd
import os
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

# Step 1: File Exploration
# -------------------------
# Define the exact path to your Parquet file in Google Drive
exact_path = '/content/drive/My Drive/ETH data/tx_data_loki_gr15.parquet'

# Load the Parquet file into a DataFrame
df = pd.read_parquet(exact_path)

# Remove rows where 'from_address' or 'to_address' is None
df = df[df['from_address'].notna() & df['to_address'].notna()]

# Randomly sample X% of the data (adjust the fraction as needed)
sample_df = df.sample(frac=0.10)

# Initialize a directed graph
G = nx.DiGraph()

# Loop through all rows in the DataFrame to add nodes and edges to the graph
for idx, row in sample_df.iterrows():
    G.add_edge(row['from_address'], row['to_address'],
               block_timestamp=row['block_timestamp'],
               eth_value=row['eth_value'],
               gas_used=row['gas_used'],  # Added gas_used
               tx_fee=row['tx_fee'])      # Added tx_fee

# Updating node types based on their appearance in 'from_address' and 'to_address'
# Here, the change is to use sample_df instead of df for node type assignment
for node in G.nodes():
    node_type = []
    if node in sample_df['from_address'].values:  # Changed from df to sample_df
        node_type.append('From')
    if node in sample_df['to_address'].values:    # Changed from df to sample_df
        node_type.append('To')
    if node in sample_df['EOA'].values:           # Changed from df to sample_df
        node_type.append('EOA')
    G.nodes[node]['type'] = node_type

# Initialize a dictionary to store the depth for each EOA
depth_dict = {}

# Define the maximum depth you're interested in
max_depth = 10

# Loop through all unique EOAs to calculate depth
# The change here is to use sample_df instead of df for the unique EOAs list
for eoa in sample_df['EOA'].unique():  # Changed from df to sample_df
    visited = set()
    to_explore = [(eoa, 0)]
    while to_explore:
        current_node, current_depth = to_explore.pop(0)
        if current_node in visited or current_depth > max_depth:
            continue
        if current_node not in G:  # Added this check to handle missing nodes
            continue
        visited.add(current_node)
        neighbors = list(G.successors(current_node))
        to_explore.extend((neighbor, current_depth + 1) for neighbor in neighbors)
    depth_dict[eoa] = len(visited)

# Count or list EOAs connected to other EOAs
eoa_to_eoa_count = 0
eoa_to_eoa_list = []

# Use sample_df for the unique EOAs list
for eoa in sample_df['EOA'].unique():  # Changed from df to sample_df
    for neighbor in G.successors(eoa):
        if 'EOA' in G.nodes[neighbor].get('type', []):  # Changed the condition to check for 'EOA' in the list
            eoa_to_eoa_count += 1
            eoa_to_eoa_list.append((eoa, neighbor))

print(f"Number of EOA to EOA connections: {eoa_to_eoa_count}")
print(f"List of EOA to EOA connections: {eoa_to_eoa_list}")

# Convert depth dictionary to DataFrame for easier manipulation and plotting
depth_df = pd.DataFrame(list(depth_dict.items()), columns=['EOA', 'Depth'])

# Basic statistics on depth
print("Depth statistics:")
print(depth_df['Depth'].describe())

# Step 3: Basic EDA
# -----------------
# View first few rows
print("First few rows of the DataFrame:")
print(df.head())

# Number of unique EOA, From and To
unique_eoas = df['EOA'].nunique()
print(f"Number of unique EOAs: {unique_eoas}")

unique_from=df['from_address'].nunique()
print(f"Number of unique from addresses: {unique_from}")

unique_to=df['to_address'].nunique()
print(f"Number of unique to addresses: {unique_to}")

# Summary statistics
print("Summary statistics:")
print(df.describe())

# Measure depth of transactions for each EOA
depth_dict = {}  # Initialize a dictionary to store the depth for each EOA
max_depth = 5  # Replace with the maximum depth you're interested in

# Convert depth dictionary to DataFrame for easier manipulation and plotting
depth_df = pd.DataFrame(list(depth_dict.items()), columns=['EOA', 'Depth'])

# Basic statistics on depth
print("Depth statistics:")
print(depth_df['Depth'].describe())

# Convert depth dictionary to DataFrame for easier manipulation and plotting
depth_df = pd.DataFrame(list(depth_dict.items()), columns=['EOA', 'Depth'])

# Basic statistics on depth
print("Depth statistics:")
print(depth_df['Depth'].describe())

# Calculate centrality metrics for all nodes
degree_centrality = nx.degree_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
# eigenvector_centrality = nx.eigenvector_centrality(G)

# Create dictionaries to store centrality metrics for the two groups
eoa_to_eoa_centrality = {}
other_eoa_centrality = {}

# Populate the dictionaries
for eoa in sample_df['EOA'].unique():
    if eoa in G:
        centrality_metrics = {
            'degree': degree_centrality.get(eoa, 0),
            'closeness': closeness_centrality.get(eoa, 0),
            'betweenness': betweenness_centrality.get(eoa, 0),
        }

        if eoa in [e[0] for e in eoa_to_eoa_list]:
            eoa_to_eoa_centrality[eoa] = centrality_metrics
        else:
            other_eoa_centrality[eoa] = centrality_metrics

# Create DataFrames for the two groups
eoa_to_eoa_df = sample_df[sample_df['EOA'].isin([e[0] for e in eoa_to_eoa_list])]
other_eoa_df = sample_df[~sample_df['EOA'].isin([e[0] for e in eoa_to_eoa_list])]

# Calculate average transaction sizes and their variance
avg_size_eoa_to_eoa = eoa_to_eoa_df['eth_value'].mean()
var_size_eoa_to_eoa = eoa_to_eoa_df['eth_value'].var()

avg_size_other_eoa = other_eoa_df['eth_value'].mean()
var_size_other_eoa = other_eoa_df['eth_value'].var()

# Calculate mean and standard deviation for each group
eoa_to_eoa_mean = eoa_to_eoa_df.mean()
eoa_to_eoa_std = eoa_to_eoa_df.std()

other_eoa_mean = other_eoa_df.mean()
other_eoa_std = other_eoa_df.std()

# Make sure that all the arrays have the same length by taking only the common metrics
common_metrics = set(eoa_to_eoa_mean.index) & set(eoa_to_eoa_std.index) & set(other_eoa_mean.index) & set(other_eoa_std.index)

# Filter the Series objects to include only the common metrics
eoa_to_eoa_mean = eoa_to_eoa_mean[common_metrics]
eoa_to_eoa_std = eoa_to_eoa_std[common_metrics]
other_eoa_mean = other_eoa_mean[common_metrics]
other_eoa_std = other_eoa_std[common_metrics]

# Convert dictionaries to DataFrames for easier manipulation
eoa_to_eoa_centrality_df = pd.DataFrame.from_dict(eoa_to_eoa_centrality, orient='index')
other_eoa_centrality_df = pd.DataFrame.from_dict(other_eoa_centrality, orient='index')

# Create a summary DataFrame
summary_df = pd.DataFrame({
    'Metric': list(common_metrics),
    'EOA_to_EOA_Mean': eoa_to_eoa_mean.values,
    'EOA_to_EOA_Std': eoa_to_eoa_std.values,
    'Other_EOA_Mean': other_eoa_mean.values,
    'Other_EOA_Std': other_eoa_std.values
})

# Display the summary DataFrame
print(summary_df)

print(f"EOA to EOA Avg Size: {avg_size_eoa_to_eoa}, Variance: {var_size_eoa_to_eoa}")
print(f"Other EOA Avg Size: {avg_size_other_eoa}, Variance: {var_size_other_eoa}")

# Calculate mean and standard deviation for centrality metrics
eoa_to_eoa_centrality_mean = eoa_to_eoa_centrality_df.mean()
eoa_to_eoa_centrality_std = eoa_to_eoa_centrality_df.std()

other_eoa_centrality_mean = other_eoa_centrality_df.mean()
other_eoa_centrality_std = other_eoa_centrality_df.std()

# Create a summary DataFrame for centrality metrics
centrality_summary_df = pd.DataFrame({
    'Centrality_Metric': list(eoa_to_eoa_centrality_mean.index),
    'EOA_to_EOA_Mean': eoa_to_eoa_centrality_mean.values,
    'EOA_to_EOA_Std': eoa_to_eoa_centrality_std.values,
    'Other_EOA_Mean': other_eoa_centrality_mean.values,
    'Other_EOA_Std': other_eoa_centrality_std.values
})

# Display the summary DataFrame for centrality metrics
print("Summary statistics for centrality metrics:")
print(centrality_summary_df)

# Check for missing values
print("Missing values:")
print(df.isnull().sum())

def df_to_markdown(df):
    fmt = ['---' for _ in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    return df_formatted.to_markdown(index=False)

# Convert summary DataFrame to Markdown
summary_markdown = df_to_markdown(summary_df)
print(summary_markdown)

# Convert centrality summary DataFrame to Markdown
centrality_summary_markdown = df_to_markdown(centrality_summary_df)
print(centrality_summary_markdown)


# Data distribution (use histograms or boxplots)
# sns.pairplot(df)
# plt.show()

# Save Processed Data
# ---------------------------
# Save the DataFrame as a new Parquet file
processed_file_path = '/content/drive/My Drive/ETH data/Sybildata_first.parquet'
df.to_parquet(processed_file_path)


Mounted at /content/drive
Number of EOA to EOA connections: 1080
List of EOA to EOA connections: [('...')]
Depth statistics:
count     5524.000000
mean       692.644642
std       2042.141880
min          1.000000
25%          4.000000
50%         10.000000
75%         28.250000
max      12615.000000
Name: Depth, dtype: float64
First few rows of the DataFrame:
                                             tx_hash  \
0  0x2329aa5420586a892f770ea1d9f7644644f9297ca6e1...   
1  0x8c05e94628693ee334513d9cc1bc7376ae9158abb496...   
2  0x76d5e123c2be36c5299a6141670e4cacf4f2a9853455...   
3  0xc35a665915e165e7053b30a73e9bdbe7a1a4a9406224...   
4  0xe0df683a17c11fc3fb4c4483e2461e84aee5f77948dc...   

            block_timestamp                                from_address  \
0  2023-01-15T05:24:59.000Z  0x000000000f709dfe4346b80009b8a5197f79aa14   
1  2022-11-15T02:15:35.000Z  0x4976a4a02f38326660d17bf34b431dc6e2eb2327   
2  2023-01-12T07:10:23.000Z  0x000000000f709dfe4346b80009b8a5197f79aa14   
3

<ipython-input-1-5931faed1b30>:167: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  eoa_to_eoa_mean = eoa_to_eoa_df.mean()
<ipython-input-1-5931faed1b30>:168: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  eoa_to_eoa_std = eoa_to_eoa_df.std()
<ipython-input-1-5931faed1b30>:170: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence

        Metric  EOA_to_EOA_Mean  EOA_to_EOA_Std  Other_EOA_Mean  Other_EOA_Std
0    gas_limit    180116.235420   316699.456630   157246.188201  296496.874150
1  __row_index     29017.541359    22222.619107    26197.684932   18819.650389
2       tx_fee         0.006356        0.025496        0.004206       0.017598
3     gas_used    119994.319426   196127.038276   109111.522737  195550.964858
4    eth_value         0.355591        4.540944        0.211512       2.984622
EOA to EOA Avg Size: 0.3555906763898585, Variance: 20.62017603745725
Other EOA Avg Size: 0.2115119233278926, Variance: 8.907969856127256
Summary statistics for centrality metrics:
  Centrality_Metric  EOA_to_EOA_Mean  EOA_to_EOA_Std  Other_EOA_Mean  \
0            degree         0.001240        0.001402        0.000389   
1         closeness         0.002259        0.004670        0.001225   
2       betweenness         0.000008        0.000025        0.000001   

   Other_EOA_Std  
0       0.000510  
1       0.003686  


In [8]:
# Aggregating transactions

# Importing necessary libraries
import pandas as pd
import functools

from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Specify the file path
file_path = '/content/drive/My Drive/ETH data/Sybildata_first.parquet'

# Read the Parquet file into a DataFrame
df = pd.read_parquet(file_path)


def aggregate_transactions(df, save_path=None):
    """
    Aggregates transactions by combining similar transactions within a time frame.
    Parameters:
        df (DataFrame): A DataFrame containing transaction data.
    Returns:
        eoa2seq_agg: A dictionary containing lists of aggregated transactions.
    """
    # Initialize the dictionary to store aggregated transactions
    eoa2seq_agg = {}

    # Iterate over unique EOAs
    for eoa in df['EOA'].unique():

        # Initialize lists for IN and OUT transactions
        in_list = []
        out_list = []

        # Filter the DataFrame to only include transactions for the current EOA
        eoa_df = df[df['EOA'] == eoa]

        # Sort the transactions by timestamp
        eoa_df_sorted = eoa_df.sort_values(by='block_timestamp')

        # Convert to list of dictionaries
        eoa_seq = eoa_df_sorted.to_dict('records')

        # Initialize a list to store the aggregated transactions for this EOA
        seq_tmp = []
        seq_tmp.append(eoa_seq[0])

        # Loop to aggregate transactions
        for i in range(1, len(eoa_seq)):
            l_trans = eoa_seq[i]  # latter transaction
            f_trans = seq_tmp[-1]  # former transaction (last in seq_tmp)
            l_time = pd.to_datetime(l_trans['block_timestamp'])
            f_time = pd.to_datetime(f_trans['block_timestamp'])
            delta_time = (l_time - f_time).seconds

            # Determine the direction (IN or OUT)
            direction = "IN" if l_trans['from_address'] == eoa else "OUT"

            # Add direction to transaction dictionaries
            l_trans['direction'] = direction
            f_trans['direction'] = direction

            # If the 'to_address' and the time difference satisfy the condition, aggregate the transactions
            if f_trans['to_address'] == l_trans['to_address'] and delta_time <= 86400 * 3:
                seq_tmp[-1]['eth_value'] += l_trans['eth_value']
            else:
                seq_tmp.append(l_trans)

        # Add the aggregated transactions to the dictionary
        eoa2seq_agg[eoa] = seq_tmp

    # Save list of dictionaries to disk if save_path is provided
    if save_path:
        with open(save_path, 'wb') as f:
            pickle.dump(eoa2seq_agg, f)

    return eoa2seq_agg

# Run the `aggregate_transactions` function and store the result
eoa2seq_agg = aggregate_transactions(df, save_path='/content/drive/My Drive/ETH data/aggregated_trans.pkl')

# The aggregated transactions are now stored in `eoa2seq_agg` and optionally saved to disk.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                                             tx_hash  \
0  0x2329aa5420586a892f770ea1d9f7644644f9297ca6e1...   
1  0x8c05e94628693ee334513d9cc1bc7376ae9158abb496...   
2  0x76d5e123c2be36c5299a6141670e4cacf4f2a9853455...   
3  0xc35a665915e165e7053b30a73e9bdbe7a1a4a9406224...   
4  0xe0df683a17c11fc3fb4c4483e2461e84aee5f77948dc...   

            block_timestamp                                from_address  \
0  2023-01-15T05:24:59.000Z  0x000000000f709dfe4346b80009b8a5197f79aa14   
1  2022-11-15T02:15:35.000Z  0x4976a4a02f38326660d17bf34b431dc6e2eb2327   
2  2023-01-12T07:10:23.000Z  0x000000000f709dfe4346b80009b8a5197f79aa14   
3  2023-01-13T11:23:11.000Z  0x000000000f709dfe4346b80009b8a5197f79aa14   
4  2023-01-12T14:46:35.000Z  0x000000000f709dfe4346b80009b8a5197f79aa14   

                                   to_address  gas_limit  gas_used    tx_fee  \
0  

ValueError: ignored

In [ ]:
# creating a function for a prettier table

from IPython.display import display, HTML

def display_styled_dataframe(df):
    """
    Display a DataFrame with formatting in Jupyter Notebook
    """
    float_columns = df.select_dtypes(include=['float64']).columns
    format_dict = {col: "{:.2}" for col in float_columns}
    styled_df = df.style.format(format_dict)
    display(styled_df)

# Using the function
display_styled_dataframe(summary_df) # summary
display_styled_dataframe(centrality_summary_df) # centrality summary


,Metric,EOA_to_EOA_Mean,EOA_to_EOA_Std,Other_EOA_Mean,Other_EOA_Std
0,gas_limit,1.8e+05,3.2e+05,1.6e+05,3e+05
1,__row_index,2.9e+04,2.2e+04,2.6e+04,1.9e+04
2,tx_fee,0.0064,0.025,0.0042,0.018
3,gas_used,1.2e+05,2e+05,1.1e+05,2e+05
4,eth_value,0.36,4.5,0.21,3.0


,Centrality_Metric,EOA_to_EOA_Mean,EOA_to_EOA_Std,Other_EOA_Mean,Other_EOA_Std
0,degree,0.0012,0.0014,0.00039,0.00051
1,closeness,0.0023,0.0047,0.0012,0.0037
2,betweenness,7.6e-06,2.5e-05,1.4e-06,9.3e-06


Centrality Metrics
1. Degree Centrality
Degree Centrality measures the number of edges a node has. In the context of Ethereum transactions, it indicates how many different addresses an EOA is interacting with. A higher degree centrality signifies that the EOA is involved in more transactions, either as a sender or a receiver.

2. Closeness Centrality
Closeness Centrality gauges how close a node is to all other nodes in the network, based on the shortest paths. For every pair of nodes, you find the shortest path between them and then average those lengths. In the Ethereum network, a lower average length means the EOA can reach other addresses through fewer hops, making it more central in the network.

3. Betweenness Centrality
Betweenness Centrality quantifies how often a node appears on the shortest paths between other nodes. In this context, a higher betweenness centrality indicates that the EOA acts as a kind of "bridge" within the network, connecting various parts of the Ethereum ecosystem.

In [6]:
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Specify the file path
file_path = '/content/drive/My Drive/ETH data/aggregated_trans.parquet'

# Read the Parquet file into a DataFrame
df = pd.read_parquet(file_path)

# Display the first few rows of the DataFrame
print(df.head())

def aggregate_and_bucketize_transactions(df):
    # Initialize a dictionary to store aggregated transactions
    eoa2seq_agg = {}

    for eoa in df['from_address'].unique():
        eoa_df = df[df['from_address'] == eoa]
        eoa_df_sorted = eoa_df.sort_values(by='block_timestamp')

        eoa_seq = []
        last_trans = eoa_df_sorted.iloc[0].to_dict()

        for i in range(1, len(eoa_df_sorted)):
            current_trans = eoa_df_sorted.iloc[i].to_dict()
            delta_time = (current_trans['block_timestamp'] - last_trans['block_timestamp']).seconds

            if last_trans['to_address'] == current_trans['to_address'] and delta_time <= 86400 * 3:
                last_trans['eth_value'] += current_trans['eth_value']
                last_trans['count'] += 1  # Assuming each row is a unique transaction

                # Bucketize `eth_value`
                eth_value = last_trans['eth_value']
                if eth_value == 0:
                    eth_value_bucket = 1
                elif eth_value <= 591:
                    eth_value_bucket = 2
                elif eth_value <= 6195:
                    eth_value_bucket = 3
                elif eth_value <= 21255:
                    eth_value_bucket = 4
                elif eth_value <= 50161:
                    eth_value_bucket = 5
                elif eth_value <= 100120:
                    eth_value_bucket = 6
                elif eth_value <= 208727:
                    eth_value_bucket = 7
                elif eth_value <= 508961:
                    eth_value_bucket = 8
                elif eth_value <= 1360574:
                    eth_value_bucket = 9
                elif eth_value <= 6500000:
                    eth_value_bucket = 10
                elif eth_value <= 143791433950:
                    eth_value_bucket = 11
                else:
                    eth_value_bucket = 12

                # Bucketize `count`
                count = last_trans['count']
                if count == 0:
                    count_bucket = 0
                elif count == 1:
                    count_bucket = 1
                elif count == 2:
                    count_bucket = 2
                elif count == 3:
                    count_bucket = 3
                elif count == 4:
                    count_bucket = 4
                elif count == 5:
                    count_bucket = 5
                elif count == 6:
                    count_bucket = 6
                elif count == 7:
                    count_bucket = 7
                elif 8 < count <= 10:
                    count_bucket = 8
                elif 10 < count <= 20:
                    count_bucket = 9
                else:
                    count_bucket = 10

                last_trans['eth_value_bucket'] = eth_value_bucket
                last_trans['count_bucket'] = count_bucket
            else:
                eoa_seq.append(last_trans)
                last_trans = current_trans

        eoa_seq.append(last_trans)
        eoa2seq_agg[eoa] = eoa_seq

    # Account Filtering
    filtered_eoa2seq_agg = {}
    for eoa, transactions in eoa2seq_agg.items():
        if 2 <= len(transactions) <= 10000:
            filtered_eoa2seq_agg[eoa] = transactions

    return filtered_eoa2seq_agg

# Assuming df is your DataFrame with the raw transactions
# aggregated_and_bucketized_data = aggregate_and_bucketize_transactions(df)


# Assuming df is DataFrame with the raw transactions
# aggregated_and_bucketized_data = aggregate_and_bucketize_transactions(df)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: ignored